# Διαδικασία: 

Αρχικά, θα δοκιμάσουμε να υλοποιήσουμε "Kasiski Attack" για να μπορέσουμε να βρούμε (κατά πάσα πιθανότητα) το μήκος 
keylength του κλειδιού, βρίσκοντας επαναλαμβανόμενα patterns στο encrypted μήνυμα.

Έπειτα, μόλις βρούμε το μήκος του κλειδιού πρέπει να βρούμε την λέξη, πιθανόν με το "Τεστ του Friedman" και τη χρήση
του δείκτη σύμπτωσης, χωρίζοντας το encrypted μήνυμα σε keylength στήλες. 

Τέλος, βρίσκουμε εύκολα αφού έχουμε το κλειδί χρησιμοποιώντας την μέθοδο του %26 για να βρούμε το αρχικό μήνυμα.


In [ ]:
# Imports
import pandas as pd
import math
from functools import reduce
import operator

In [ ]:
with open('vigenere.txt', 'r', encoding='utf-8-sig') as file:
    coded_msg = file.read().replace('\n', '')
display(coded_msg)

In [ ]:
len(coded_msg)

In [ ]:
def kasiski_attack(text, n):
    i = 0
    count = {}
    
    for i in range(0, len(text)-(n-1)):
        #s = text[i] + text[i+1] + text[i+2] + text[i+3]
        s = text[i]
        for j in range(i+1, i+n):
            if j < len(text):
                s = s + text[j]
        if s in count:
            count[s] += 1
        else:
            count[s] = 1
        
    return count

In [ ]:
columns = ['Length','Substring', 'Frequency']
df = pd.DataFrame(columns = columns)

In [ ]:
for q in range(4,10):
    count = kasiski_attack(coded_msg, q)
    for key in count:
        if count[key] > 2:
            df = df.append({'Length': q, 'Substring': key, 'Frequency': count[key]}, ignore_index=True)
            #print ("Length:", q, " substring:", key, " freq:", count[key])

In [ ]:
la = [[] for _ in range(52)]
for index, row in df.iterrows():
    count_stars = 0
    count = 0
    start = False
    for char in coded_msg.replace(row['Substring'], "*"):
        if char == "*":
            if count != 0:
                la[index].append(count-1)
            count_stars = count_stars + 1
            start = True
            count=0
        if count_stars == row['Frequency']:
            break
        if start == True:
            count = count + 1

In [ ]:
df['Distances'] = la 

In [ ]:
def find_gcd(list):
    x = reduce(math.gcd, list)
    return x

In [ ]:
gcd_list = []
for index, row in df.iterrows():
    gcd_list.append(find_gcd(row['Distances']))

In [ ]:
df['GCD'] = gcd_list 

In [ ]:
display(df)

In [ ]:
indexNames = df[df['GCD'] < 3 ].index

df.drop(indexNames , inplace=True)
df.reset_index()

In [ ]:
n=7
#https://stackoverflow.com/questions/22571259/split-a-string-into-n-equal-parts
parts = [coded_msg[i:i+n] for i in range(0, len(coded_msg), n)]

In [ ]:
display(parts)

In [ ]:
mapping = [list(word) for word in parts]

In [ ]:
display(mapping)

In [ ]:
letters = []
for i in range(0,len(y)):
    letters.append(y[i][0])
text = ''.join(map(str, letters)) 
display(text)

In [ ]:
df_list = []
columns = ['Letter', 'Frequency']
df_letters = pd.DataFrame(columns = columns)
for j in range(0,n):
    letters = []
    for i in range(0,len(y)):
        letters.append(y[i][j])
    text = ''.join(map(str, letters))
    print(str(j))
    print(text)
    count = kasiski_attack(text, 1)
    #print(count)
    df_list.append(count)

In [ ]:
for i in range(0, n):
    print(i)
    print(df_list[i])

In [ ]:
for i in range(0, n):
    temp = max(df_list[i].items(), key=operator.itemgetter(1))[0]
    print(temp + str(': ')+str(df_list[i][temp]))

In [ ]:
data0 = []
data1 = []
data2 = []
data3 = []
data4 = []
data5 = []
data6 = []

In [ ]:
p=0
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data0.append(values)

In [ ]:
p=1
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data1.append(values)

In [ ]:
p=2
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data2.append(values)

In [ ]:
p=3
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data3.append(values)

In [ ]:
p=4
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data4.append(values)

In [ ]:
p=5
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data5.append(values)

In [ ]:
p=6
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    print(i + ": " +str(values))
    data6.append(values)

In [ ]:
data = {
    "A": 8.12,
    "B": 1.49,
    "C": 2.71,
    "D": 4.32,
    "E": 12.02,
    "F": 2.3,
    "G": 2.03,
    "H": 5.92,
    "I": 7.31,
    "J": 0.1,
    "K": 0.69,
    "L": 3.92,
    "M": 2.61,
    "N": 6.95,
    "O": 7.68,
    "P": 1.82,
    "Q": 0.11,
    "R": 6.02,
    "S": 6.28,
    "T": 9.1,
    "U": 2.88,
    "V": 1.11,
    "W": 2.09,
    "X": 0.17,
    "Y": 2.11,
    "Z": 0.07

}

In [ ]:
display(data)

In [ ]:
import string
column = list(string.ascii_uppercase)
new = pd.DataFrame(columns = column)

In [ ]:
newdf=pd.DataFrame(fulldata,columns=column)

In [ ]:
fulldata = []
fulldata.append(data)
fulldata.append(data0)
fulldata.append(data1)
fulldata.append(data2)
fulldata.append(data3)
fulldata.append(data4)
fulldata.append(data5)
fulldata.append(data6)

In [ ]:
data = {8.12, 1.49, 2.71, 4.32, 12.02, 2.3, 2.03, 5.92, 7.31, 0.1,
 0.69,
 3.92,
 2.61,
 6.95,
 7.68,
 1.82,
 0.11,
 6.02,
 6.28,
 9.1,
 2.88,
 1.11,
 2.09,
 0.17,
 2.11,
 0.07
}